In [ ]:
data_dir = joinpath(dirname(pwd()), "data")

In [ ]:
# db_path = joinpath(blastdb_dir, blast_db)

In [ ]:
sample_paths = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(joinpath(data_dir, "SRA"), join=true))
sample_paths = filter(x -> isfile(joinpath(x, "megahit", "final.contigs.fastg.gfa.fna")), sample_paths)

In [ ]:
memory=64
threads=Int(memory//8)

In [ ]:
for sample_path in sample_paths
    sample = basename(sample_path)
    target_outfile = joinpath(sample_path, "blastn", "final.contigs.fastg.gfa.fna.blastn.nt.megablast.txt")
    
    if !isfile(target_outfile)
        @show sample_path
        @show sample
        assembled_fasta = joinpath(sample_path, "megahit", "final.contigs.fastg.gfa.fna")    
        cmd = "julia $(homedir())/workspace/Mycelia/scripts/run-blastn.jl --fasta $(assembled_fasta) --blast_db nt --out_directory $(sample_path) --threads $(threads)"
        submission = 
        `sbatch 
        --mail-user=cameron.prybol@gmail.com
        --mail-type=ALL
        --error=%x-%j.err
        --output=%x-%j.out
        --partition=batch
        --account=mpsnyder
        --nodes=1
        --ntasks=1
        --time=1-00:00:00    
        --cpus-per-task=$(threads)
        --mem=$(memory)G
        --job-name=blastn
        --wrap $cmd
        `
        run(submission)
    end
end